In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QIcon


# Peice classes
class Piece:
    def __init__(self, colour):
        self.colour = colour


class Rook(Piece):
    def __init__(self, colour):
        super().__init__(colour)


class Knight(Piece):
    def __init__(self, colour):
        super().__init__(colour)


class Bishop(Piece):
    def __init__(self, colour):
        super().__init__(colour)


class Queen(Piece):
    def __init__(self, colour):
        super().__init__(colour)


class King(Piece):
    def __init__(self, colour):
        super().__init__(colour)


class Pawn(Piece):
    def __init__(self, colour):
        super().__init__(colour)
        self.firstMove = True

    def getValidMoves(self, board, x, y):
        valid_moves = []
        if self.colour == "white":
            # Check if the square in front of the pawn is empty
            if x > 0 and hasattr(board[x + 1][y], 'colour'):
                valid_moves.append((x + 1, y))
                # Check if the pawn is on its first move and if the square two squares in front of the pawn is empty
                if self.firstMove and board[x + 2][y] is None:
                    valid_moves.append((x + 2, y))
            # Check if the square to the left of the pawn is occupied by an enemy piece
            if hasattr(board[x + 1][y + 1], 'colour'):
                print(board[x + 1][y + 1])
                if x > 0 and y > 0 and (board[x + 1][y + 1].colour == "black"):
                    valid_moves.append((x + 1, y + 1))
            if hasattr(board[x + 1][y - 1], 'colour'):
                print(board[x +1][y-1])
                # Check if the square to the right of the pawn is occupied by an enemy piece
                if x > 0 and y < 7 and (board[x + 1][y - 1].colour == "black"):
                    valid_moves.append((x + 1, y - 1))
        else:
            # Check if the square in front of the pawn is empty
            if x < 7 and board[x - 1][y] is None:
                valid_moves.append((x - 1, y))
                # Check if the pawn is on its first move and if the square two squares in front of the pawn is empty
                if self.firstMove and board[x - 2][y] is None:
                    self.firstMove = False
                    valid_moves.append((x - 2, y))
            # Check if the square to the left of the pawn is occupied by an enemy piece
            if x < 7 and y > 0:
                if board[x + 1][y - 1] == "None":
                    pass
                elif board[x + 1][y - 1].colour == "white":
                    valid_moves.append((x + 1, y - 1))
            # Check if the square to the right of the pawn is occupied by an enemy piece
            if x < 7 and y < 7:
                if board[x + 1][y + 1] == "None":
                    pass
                elif board[x + 1][y + 1].colour == "white":
                    valid_moves.append((x - 1, y - 1))
        print(valid_moves, "valid moves")
        return valid_moves


class ChessBoard:
    def __init__(self):
        self.moveCount = 0
        self.playerTurn = "White"

        self.board = [[None for x in range(8)] for y in range(8)]
        self.buttons = {}
        self.selectedButton = None
        self.highlightedSquares = []

        # Create white pieces
        self.board[0][0] = Rook("white")
        self.board[0][1] = Knight("white")
        self.board[0][2] = Bishop("white")
        self.board[0][3] = Queen("white")
        self.board[0][4] = King("white")
        self.board[0][5] = Bishop("white")
        self.board[0][6] = Knight("white")
        self.board[0][7] = Rook("white")
        for i in range(8):
            self.board[1][i] = Pawn("white")

        # Create black pieces
        self.board[7][0] = Rook("black")
        self.board[7][1] = Knight("black")
        self.board[7][2] = Bishop("black")
        self.board[7][3] = Queen("black")
        self.board[7][4] = King("black")
        self.board[7][5] = Bishop("black")
        self.board[7][6] = Knight("black")
        self.board[7][7] = Rook("black")
        for i in range(8):
            self.board[6][i] = Pawn("black")

    # Draw the board squares
    def drawSquare(self, x, y):
        button = QPushButton()
        button.setFixedSize(100, 100)
        if (x + y) % 2 == 0:
            button.setProperty("class", "white")
            button.setStyleSheet("background-color: white; border: None")
        else:
            button.setProperty("class", "black")
            button.setStyleSheet("background-color: green; border: None")

        self.buttons[f"{x},{y}"] = button
        button.setObjectName(
            f"{x},{y}"
        )  # Set the objectName property to the square coordinates

        # Connect the clicked signal of the button to a slot
        button.clicked.connect(lambda: self.checkMove(self.board[x][y], f"{x},{y}"))

        return button

    # Slot that checks if a piece and a square have been clicked, and calls the movePeice function
    def checkMove(self, piece, square):
        if piece is not None and square is not None:
            x, y = [int(i) for i in square.split(",")]
            if self.selectedButton is None:
                self.selectedButton = self.buttons[square]
                self.selectedButton.setStyleSheet(
                    "background-color: yellow; border: None"
                )
                self.highlightSquares(piece, self.getValidMoves(self.board, x, y))
            else:
                current_x, current_y = [
                    int(i) for i in self.selectedButton.objectName().split(",")
                ]
                self.movePiece(piece, current_x, current_y, x, y)
                self.selectedButton.setStyleSheet(
                    "background-color: None; border: None"
                )
                self.selectedButton = None
                self.highlightSquares(
                    piece, [x, y]
                )  # Pass new_x and new_y to highlightSquares
                self.drawSquare(
                    current_x, current_y
                )  # Reset the background color of the previously selected button

    # Draw the chess board
    def drawBoard(self, layout):
        for x in range(8):
            for y in range(8):
                button = self.drawSquare(x, y)
                layout.addWidget(button, x, y)

        # Check if there is a piece on the square and call drawPiece
        for x in range(8):
            for y in range(8):
                if self.board[x][y] is not None:
                    self.drawPiece(self.buttons[f"{x},{y}"], self.board[x][y])

    # Draw image on a button
    def drawPiece(self, button, piece):
        if piece is not None and piece.__class__.__name__ != "None":
            icon = QIcon(f"media/{piece.colour}/{piece.__class__.__name__}.svg")
            button.setIcon(icon)
            button.setIconSize(button.size())
        # remove icon
        else:
            button.setIcon(QIcon())
            button.setIconSize(button.size())

    def getValidMoves(self, board, x, y):
        x = int(x)
        y = int(y)
        if self.board[x][y] is not None:
            return self.board[x][y].getValidMoves(board, x, y)
            print(self.board[x][y].getValidMoves(board, x, y))
        else:
            return []

    def highlightSquares(self, piece, squares):
        # Remove existing highlighted squares
        for button in self.highlightedSquares:
            # check is the square is meant to be green or white
            if (
                int(button.objectName().split(",")[0])
                + int(button.objectName().split(",")[1])
            ) % 2 == 0:
                button.setStyleSheet("background-color: white; border: None")
            else:
                button.setStyleSheet("background-color: green; border: None")

            button.clicked.disconnect()
        self.highlightedSquares = []

        for square in squares:
            if square[0] >= 0 and square[0] <= 7 and square[1] >= 0 and square[1] <= 7:
                button = self.buttons[f"{square[0]},{square[1]}"]
                self.highlightedSquares.append(button)
                button.setStyleSheet("background-color: blue; border: None")
                button.clicked.connect(
                    lambda _, piece=piece, current_x=self.selectedButton.objectName().split(
                        ","
                    )[
                        0
                    ], current_y=self.selectedButton.objectName().split(
                        ","
                    )[
                        1
                    ], new_x=square[
                        0
                    ], new_y=square[
                        1
                    ]: self.movePiece(
                        piece, current_x, current_y, new_x, new_y
                    )
                )

    def movePiece(self, piece, current_x, current_y, new_x, new_y):
        current_x = int(current_x)
        current_y = int(current_y)
        print(f"{self.playerTurn}player turn: {self.moveCount} move count")
        if piece is not None and piece.colour == self.playerTurn:
            valid_moves = self.getValidMoves(self.board, current_x, current_y)
            if (new_x, new_y) in valid_moves:
                self.board[new_x][new_y] = piece
                self.board[current_x][current_y] = None
                piece.x = new_x
                piece.y = new_y
                self.drawPiece(self.buttons[f"{new_x},{new_y}"], piece)
                if self.buttons[f"{current_x},{current_y}"].icon() is not None:
                    self.drawPiece(self.buttons[f"{current_x},{current_y}"], None)
                    # increase move count
                    self.moveCount += 1
                    self.playerTurn = "White" if self.moveCount % 2 == 0 else "Black"
                self.highlightSquares(
                    piece, self.getValidMoves(self.board, new_x, new_y)
                )
            else:
                print("invalid move")
        else:
            print("invalid move")
            self.highlightSquares(None, [])


# UI
class MainWindow(QWidget):
    def __init__(self):
        QWidget.__init__(self)
        self.windowTitle = "Chess"

        # Create layout
        layout = QGridLayout()
        self.setLayout(layout)

        # Create chess board
        board = ChessBoard()
        board.drawBoard(layout)


if __name__ == "__main__":
    app = QApplication([])
    window = MainWindow()
    window.show()
    app.exec()

white board
True
True
False
[] valid moves
Whiteplayer turn: 0 move count
invalid move


TypeError: 'int' object is not subscriptable